<a href="https://colab.research.google.com/github/MohitDhungana/duplicate_question_detection/blob/master/predict_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import re
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.models import load_model


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
def text_to_word_list(text):
#     gets a whole question inside text variable on which  preprocessing is done and then the question is splitted into word indices and returned
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [0]:
# manhattan distance calculation function

def manhattan_distance(left,right):
  return K.exp(-K.sum(K.abs(left-right),axis=1, keepdims=True))

In [0]:
# import vocabulary dictionary and parse it in python

dict_path='/content/drive/My Drive/vocabulary_dictionary.json'

# read file
with open(dict_path, 'r') as my_file:
    data=my_file.read()

# parse file
vocab_dictionary = json.loads(data)

In [0]:
# function to index tokenized words using training vocabulary dictionary

def word_indexer(question):
  ques_index=[]

  for word in text_to_word_list(question):
    if word in vocab_dictionary:
      ques_index.append(vocab_dictionary[word])      
  return ques_index

In [0]:
# load trained model 
model_path='/content/drive/My Drive/85DQD2.h5'
new_model = load_model(model_path)
# new_model.summary()

In [24]:
# MAIN SECTION

# sample question
ques1='How YouTube pay?'
ques2='When can Youtube pay?'

# preprocess and tokenize sample questions 
q1=word_indexer(ques1)
# print(q1)
q2=word_indexer(ques2)
# print(q2)


question_list=[q1,q2]
question_list=pad_sequences(question_list,maxlen = 50)

que1=np.asarray(question_list[0])
que2=np.asarray(question_list[1])

# predict sample questions on saved model

pred=new_model.predict([[que1],[que2]])

if pred>=0.5:
  print("Duplicate ",pred)
else:
  print("Not Duplicate",pred)

Not Duplicate [[0.40657452]]
